In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import ToTensor
import torch.nn.functional as F

In [2]:
dataset =  torchvision.datasets.MNIST('/data', train= True , transform= ToTensor(), download= True ) 

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /data/MNIST/raw/train-images-idx3-ubyte.gz to /data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /data/MNIST/raw/train-labels-idx1-ubyte.gz to /data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /data/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/MNIST/raw



In [3]:
train , test = random_split(dataset , [50000,10000])

In [4]:
train_dl , test_dl = DataLoader(train , 32 , True),DataLoader(test , 32 , True)

In [5]:
len(train_dl)

1563

### creating a model

In [6]:
class model_mnist(torch.nn.Module):
  """nn with one hidden layer"""
  def __init__(self,input , hidden_layar_size, output):
    super().__init__()
    self.linear1 = torch.nn.Linear(input,hidden_layar_size)
    self.linear2 = torch.nn.Linear(hidden_layar_size, output)


  def forward(self,xb):
    xb = xb.view(xb.shape[0],-1)
    out = self.linear1(xb)
    out = F.relu(out)
    out = self.linear2(out)
    return out

In [7]:
model = model_mnist((28*28),5, 10)

In [8]:
for xb,yb in train:
  preds = model.forward(xb)
  break

In [9]:
xb.shape[0]

1

In [10]:
preds

tensor([[-0.1479, -0.0937,  0.3951,  0.2559, -0.2033,  0.0273,  0.4421, -0.0374,
          0.4420,  0.1722]], grad_fn=<AddmmBackward0>)

Try it 

In [11]:
for xb , yb in train_dl:
  output = model.forward(xb)
  loss = F.cross_entropy(output,yb)
  preds = F.softmax(output)
  print( "loss is " , loss)
  print(preds)
  print(torch.sum(preds[0]))
  break

loss is  tensor(2.3240, grad_fn=<NllLossBackward0>)
tensor([[0.0669, 0.0722, 0.1408, 0.0999, 0.0786, 0.0910, 0.1271, 0.0821, 0.1214,
         0.1201],
        [0.0724, 0.0775, 0.1280, 0.0991, 0.0739, 0.0851, 0.1304, 0.0805, 0.1326,
         0.1206],
        [0.0750, 0.0806, 0.1252, 0.1026, 0.0743, 0.0865, 0.1287, 0.0800, 0.1303,
         0.1168],
        [0.0713, 0.0773, 0.1356, 0.1064, 0.0794, 0.0937, 0.1240, 0.0812, 0.1177,
         0.1134],
        [0.0690, 0.0785, 0.1294, 0.1012, 0.0710, 0.0906, 0.1329, 0.0812, 0.1320,
         0.1143],
        [0.0715, 0.0731, 0.1321, 0.0990, 0.0745, 0.0824, 0.1316, 0.0818, 0.1338,
         0.1202],
        [0.0751, 0.0795, 0.1265, 0.1002, 0.0736, 0.0889, 0.1277, 0.0802, 0.1307,
         0.1176],
        [0.0727, 0.0826, 0.1274, 0.1028, 0.0760, 0.0929, 0.1257, 0.0794, 0.1231,
         0.1174],
        [0.0695, 0.0736, 0.1316, 0.0955, 0.0735, 0.0831, 0.1325, 0.0812, 0.1351,
         0.1245],
        [0.0716, 0.0791, 0.1287, 0.0997, 0.0749, 0.0886, 

<ipython-input-11-a265588f7b90>:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds = F.softmax(output)


### GPU

In [12]:
def get_availble_device():
  if torch.cuda.is_available():
    print(torch.device("cuda"))
  else:
    print(torch.device("cpu"))

In [13]:
get_availble_device()

cuda


build to device function

In [70]:
def to_device(data, device):
  if isinstance(data, (list,tuple)):
    return[to_device(d,device) for d in data ]
  return data.to(device)

In [35]:
get_availble_device()

cuda


In [44]:
for xb,yb in train_dl:
  to_device(xb,"cuda")

SyntaxError: ignored

In [42]:
xb.shape

torch.Size([16, 1, 28, 28])

In [100]:
class DeviceDataLoader():
  def __init__(self,dl , device):
    self.dl = dl
    self.device = device
  def __iter__(self):
    for b in self.dl:
      yield to_device(b, self.device)

  def __len__(self):
    return len(self.dl)


In [101]:
train_data = DeviceDataLoader(train_dl, "cuda")
valid_data = DeviceDataLoader(test_dl, "cuda")

In [73]:
train_data.device

'cuda'

the whole thing again

In [125]:
dataset =  torchvision.datasets.MNIST('/data', train= True , transform= ToTensor(), download= True ) 

In [126]:
train,valid = random_split(dataset, [50000,10000])
train_dl, valid_dl = DataLoader(train, 100) , DataLoader(test, 100)

In [134]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [135]:
train_data = DeviceDataLoader(train_dl, "cuda")

In [127]:
def evaluate(model, val_loader):
    """Evaluate the model's performance on the validation set"""
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [179]:
class model_mnist(torch.nn.Module):
  """nn with one hidden layer"""
  def __init__(self,input , hidden_layar_size, output):
    super().__init__()
    self.linear1 = torch.nn.Linear(input,hidden_layar_size)
    self.linear2 = torch.nn.Linear(hidden_layar_size, output)


  def forward(self,xb):
    xb = xb.view(xb.shape[0],-1)
    out = self.linear1(xb)
    out = F.relu(out)
    out = self.linear2(out)
    return out
  def training_step(self, batch):
    images , labels = batch
    out = self(images)
    loss = F.cross_entropy(out,labels)
    acc = accuracy(torch.argmax(out,dim = 1),labels)
    return loss




In [ ]:
opt = torch.optim.SGD(model.parameters(),0.0001)
for epoch in range(5):
  for batch in train_data:
    loss = model.training_step(batch)
    loss.backward()
    opt.step()
    opt.zero_grad()
    print(accuracy(torch.argmax(out,1),labels))
    


In [160]:
opt.step()
opt.zero_grad()

In [180]:
def accuracy(l1,l2):
  return(l1==l2).sum()/len(l1)

In [196]:
def fit(dl,epochs, model , optimizer,lr):
  losses = []
  opt = optimizer(model.parameters(), lr)
  for epoch in range(epochs):
    for batch in dl:
      loss = model.training_step(batch)
      loss.backward()
     
      opt.step()
      opt.zero_grad()
      
      losses.append(loss)
      #accuracy.append(acc)
  history= {"accuracy":accuracy, "loss":losses}
  return loss



In [182]:
# hyberparameters
INPUT_SIZE = 28*28
HIDDEN_SIZE = 32
OUTPUT_SIZE = 10
LR =  1e-3

In [183]:
model = model_mnist(INPUT_SIZE, HIDDEN_SIZE,OUTPUT_SIZE)

In [184]:
# moving model to device
model = to_device(model, "cuda")

In [197]:
fit(train_data,5,model,torch.optim.SGD,1e-3)

tensor(0.7367, device='cuda:0', grad_fn=<NllLossBackward0>)

In [151]:
len()

2